# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from vacation_api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,279.35,92,100,3.89,SJ,1722470817
1,1,port-aux-francais,-49.3500,70.2167,278.00,98,100,12.57,TF,1722470817
2,2,bama,11.5221,13.6856,296.07,95,100,2.52,NG,1722470818
3,3,adamstown,-25.0660,-130.1015,295.43,87,23,9.54,PN,1722470818
4,4,vila franca do campo,37.7167,-25.4333,295.38,83,43,1.54,PT,1722470818


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.query("`Max Temp` <= 299")

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,279.35,92,100,3.89,SJ,1722470817
1,1,port-aux-francais,-49.3500,70.2167,278.00,98,100,12.57,TF,1722470817
2,2,bama,11.5221,13.6856,296.07,95,100,2.52,NG,1722470818
3,3,adamstown,-25.0660,-130.1015,295.43,87,23,9.54,PN,1722470818
4,4,vila franca do campo,37.7167,-25.4333,295.38,83,43,1.54,PT,1722470818


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()
hotel_df = hotel_df[["City", "Country", "Humidity", "Lat", "Lng"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Humidity,Lat,Lng,Hotel Name
0,olonkinbyen,SJ,92,70.9221,-8.7187,
1,port-aux-francais,TF,98,-49.3500,70.2167,
2,bama,NG,95,11.5221,13.6856,
3,adamstown,PN,87,-25.0660,-130.1015,
4,vila franca do campo,PT,83,37.7167,-25.4333,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
limit = 20
radius = 50

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
    "limit": limit,
    "radius": radius,
    "filter": f"circle:{lng},{lat},{radius}",
    "bias": f"proximity:{lng},{lat}",
    "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
olonkinbyen - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
bama - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
berezovyy - nearest hotel: No hotel found
floro - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
bouna - nearest hotel: No hotel found
lys'va - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
pisco - nearest hotel: No hotel 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    
)

# Display the map
map_plot_2 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)